In [90]:
import pandas as pd
import numpy as np
import json
import re
import nltk

from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Read data file into a python array
with open('../data/test_tip.json', 'rb') as f:
    bus_data = f.readlines()

# remove the trailing "\n" from each line
bus_data = map(lambda x: x.rstrip(), bus_data)
# put individual business JSON objects into list
data_json = "[" + ','.join(bus_data) + "]"

# Create pandas df
bus_df = pd.read_json(data_json)

In [91]:
bus_df.head()

,business_id,date,likes,text,type,user_id
0,cE27W9VPgO88Qxe4ol6y_g,2013-04-18,0,Don't waste your time.,tip,-6rEfobYjMxpUWLNxszaxQ
1,mVHrayjG3uZ_RLHkLj-AMg,2013-01-06,1,Your GPS will not allow you to find this place...,tip,EZ0r9dKKtEGVx2CdnowPCw
2,KayYbHCt-RkbGcPdGOThNg,2013-12-03,0,Great drink specials!,tip,xb6zEQCw9I-Gl0g06e1KsQ
3,KayYbHCt-RkbGcPdGOThNg,2015-07-08,0,"Friendly staff, good food, great beer selectio...",tip,QawZN4PSW7ng_9SP7pjsVQ
4,1_lU0-eSWJCRvNGk78Zh9Q,2015-10-25,0,Beautiful restoration.,tip,MLQre1nvUtW-RqMTc4iC9A


In [92]:
bus_df.text[1]

u'Your GPS will not allow you to find this place. Put Rankin police department in instead. They are directly across the street.'

In [93]:
# Helper functions for normalising text data

# Convert all words to lowercase, remove punctuation, tokenise and stem
# and remove stopwords, threshold = 10%
def norm_corpus(document):

    # lowercase and remove symbols
    tokenizer = RegexpTokenizer(r'\w+')
    doc_tokens = tokenizer.tokenize(document.lower())
        
    # remove stopwords
    doc_tokens = [word for word in doc_tokens if word not in stopwords.words('english')]
        
    # stem words
    stemmer = SnowballStemmer("english")
    doc_stem = [stemmer.stem(word) for word in doc_tokens]
        
    # make tokenised text one string
    norm_doc = " ".join(doc_stem)
    
    return norm_doc

In [94]:
# Helper functions for vectorise normalised data

In [95]:
# Vectorise keywords from normalised text to vector including only nouns and adjectives
def review_vector(norm_doc):

    # select all words categorised as nouns or adjectives
    # loop through each string i.e. review in the df column
    review_keyword_list = []
    doc = nltk.word_tokenize(norm_doc)

    # create tuple for each word in list: (word, tag)
    token_category = nltk.pos_tag(doc)  

    for word, tag in token_category:   
            
        # nouns
        if (tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS'):
            review_keyword_list.append(word)
                
        # adjectives
        elif (tag == 'JJ' or tag == 'JJS' or tag == 'JJP' or tag == 'JJPS'):
            review_keyword_list.append(word)
        
        else:
            pass     
        
    review_keywords = " ".join(review_keyword_list)
        
    # vectorise string
    WORD = re.compile(r'\w+')
    review_vector = Counter(WORD.findall(review_keywords))
    
    
    return review_vector

In [118]:
output_df = bus_df[['business_id', 'user_id', 'date']]
output_df['tip'] = bus_df.text.apply(lambda x: norm_corpus(x))
print "tip text normalised, next: vectorise"
output_df.tip = output_df.tip.apply(lambda x: review_vector(x))
output_df.head()

tip text normalised, next: vectorise


/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,business_id,user_id,date,tip
0,cE27W9VPgO88Qxe4ol6y_g,-6rEfobYjMxpUWLNxszaxQ,2013-04-18,"{u'wast': 1, u'time': 1}"
1,mVHrayjG3uZ_RLHkLj-AMg,EZ0r9dKKtEGVx2CdnowPCw,2013-01-06,"{u'depart': 1, u'polic': 1, u'rankin': 1, u'di..."
2,KayYbHCt-RkbGcPdGOThNg,xb6zEQCw9I-Gl0g06e1KsQ,2013-12-03,"{u'great': 1, u'special': 1}"
3,KayYbHCt-RkbGcPdGOThNg,QawZN4PSW7ng_9SP7pjsVQ,2015-07-08,"{u'great': 1, u'good': 1, u'food': 1, u'beer':..."
4,1_lU0-eSWJCRvNGk78Zh9Q,MLQre1nvUtW-RqMTc4iC9A,2015-10-25,"{u'beauti': 1, u'restor': 1}"


In [102]:
# output_df = bus_df[['business_id', 'user_id', 'date', 'stars', 'votes']]
# output_df['review'] = norm_corpus(bus_df.text)
# print "review text normalised, next: vectorise"
# output_df.review = review_vector(output_df.review)